# RC Benchmarks

This notebook establishes all the available benchmarks for the FSPM reservoir evaluation framework.

In [1]:
import pandas as pd
import numpy as np
import sys, os

from matplotlib import pyplot as plt
sys.path.insert(1, os.path.join(sys.path[0], '../'))  # for importing local packages from src

%reload_ext autoreload
%autoreload 2 